In [1]:
import gdal
from gdalconst import GA_ReadOnly
import os
import glob
import matplotlib.pyplot as plt
from datetime import datetime
import rasterio as rio
from rasterio.warp import calculate_default_transform, reproject, Resampling

In [2]:
# usdm file directory
usdm_dir = "R:\\UnitedStates\\ForDRI_project-NEW\\USDM_Evaluation_2023\\USDM"
# iterating through all file into a list
usdm_list = []
for subdir, dir, files in os.walk(usdm_dir):
    for file in files:
        if file[-3:]=="img":
            dir_path = os.path.join(subdir, file)
            if os.path.isfile(dir_path):
                usdm_list.append(dir_path)
                
# another way to do it
# usdm_list = [f for f in glob.iglob(usdm_dir+"\\*\\*.img",recursive=True) if os.path.isfile(f)]

usdm_list.sort()

In [8]:
'''
works so slow

dst_crs = 'EPSG:4326' # target projection
odir = "R:\\UnitedStates\\ForDRI_project-NEW\\USDM_Evaluation_2023\\USDM_rpj\\"
for usdm in usdm_list:
    fname = os.path.basename(usdm)
    doy = datetime.strptime(fname.split("_")[1],"%Y%m%d").strftime("%Y%j")
    woy = int(doy[-3:])//7
    if woy < 10:
        woy_str = '0'+str(woy)
    else:
        woy_str = str(woy)
    new_name = "USDM_"+ doy[:4]+woy_str+".tif"
    with rio.open(usdm) as src:
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds)
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })
        with rio.open(odir+new_name, 'w', **kwargs) as dst:
            reproject(
                source=rio.band(src, 1),
                destination=rio.band(dst, 1),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest)
        
    print(new_name)
'''


In [23]:
ds = gdal.Open('R:\\UnitedStates\\ForDRI_project-NEW\\New_Model_2022\\ForDRI\\ForDRI\\ForDRI_200301.tif',GA_ReadOnly)# your mask raster
proj = ds.GetProjectionRef()
geot = ds.GetGeoTransform()
minx = geot[0]
maxy = geot[3]
maxx = minx + geot[1] * ds.RasterXSize
miny = maxy + geot[5] * ds.RasterYSize

In [ ]:
odir_rpj = "R:\\UnitedStates\\ForDRI_project-NEW\\USDM_Evaluation_2023\\USDM_rpj\\"

for f in usdm_list:
    fname = os.path.basename(f)
    doy = datetime.strptime(fname.split("_")[1],"%Y%m%d").strftime("%Y%j")
    woy = int(doy[-3:])//7
    if woy < 10:
        woy_str = '0'+str(woy)
    else:
        woy_str = str(woy)
    new_name = "USDM_"+ doy[:4]+woy_str+".tif"
    
    data_toreprj = gdal.Open(f, GA_ReadOnly) # data to reproject
    gdal.Warp(odir_rpj+new_name, data_toreprj,format='GTiff',outputBounds=[-125.0, 50.0, -67.0, 24.0],resampleAlg="near",dstNodata=-9999,dstSRS=proj)
    
    print(fname)

In [ ]:
# rename file
for i in range(2004,2005):
    usdm_prj_tif_list = glob.glob("R:\\UnitedStates\\ForDRI_project-NEW\\USDM_Evaluation_2023\\USDM_rpj\\USDM_"+str(i)+"*.tif")
    usdm_prj_tif_list.sort()
    usdm_prj_xml_list = glob.glob("R:\\UnitedStates\\ForDRI_project-NEW\\USDM_Evaluation_2023\\USDM_rpj\\USDM_"+str(i)+"*.xml")
    usdm_prj_xml_list.sort()
    if usdm_prj_tif_list[0].split("\\")[-1][-6:-4]=="00":
        print(usdm_prj_tif_list[0])
        for usdm_prj_tif in usdm_prj_tif_list:
            fname = os.path.basename(usdm_prj_tif)
            week = int(fname[-6:-4])+1
            if week < 10:
                week_str = "0" + str(week)
            else:
                week_str = str(week)
            new_name = str(i)+week_str+".tif"
            os.rename(usdm_prj_tif, "R:\\UnitedStates\\ForDRI_project-NEW\\USDM_Evaluation_2023\\USDM_"+new_name)
        for usdm_prj_xml in usdm_prj_xml_list:
            fname = os.path.basename(usdm_prj_xml)
            week = int(fname[-14:-12])+1
            if week < 10:
                week_str = "0" + str(week)
            else:
                week_str = str(week)
            new_name = str(i)+week_str+".tif.aux.xml"
            os.rename(usdm_prj_xml, "R:\\UnitedStates\\ForDRI_project-NEW\\USDM_Evaluation_2023\\USDM_"+new_name)

In [20]:
fname

'USDM_200400.tif.aux.xml'

In [15]:
usdm_prj_tif_list[0].split("\\")[-1]

'USDM_202200.tif'